In [1]:
import pandas as pd

In [2]:
# import pre-race csv with the club info
clubs=pd.read_csv('../data/rotg-final.csv')

clubs=clubs[['first','last','gender','age','clubname']]

# create collumns for open, masters and agegrade
clubs['open']='yes'
clubs['master'] =  clubs['age'].map(lambda x: 'no' if x < 40 else 'yes')
clubs['agegrade'] =  clubs['age'].map(lambda x: 'no' if x < 50 else 'yes')

In [3]:
# create name column
clubs['Name']=clubs['first']+" "+clubs['last']

# capitalize names
clubs['Name']=clubs['Name'].apply(lambda x: x.title())

print clubs.shape

(336, 9)


In [4]:
# load actual results
results=pd.read_fwf('../data/rotg2016.txt')

In [5]:
# in this step we use diflib to map 'results.Name' to the closest match in 'clubs.Name' this will ensure
# we have the same name in each
import difflib 
results.Name = results.Name.map(lambda x: difflib.get_close_matches(x, clubs.Name, cutoff=0.4)[0])

In [6]:
# now lets merge results with clubs
results=pd.merge(clubs, results, on='Name')
print results.shape

(305, 17)


In [7]:
# in this block we convert the time string into a timedelta and seconds column which can be eased
# for numerical operations (sorting, adding times, etc.)

import numpy as np
import datetime

# first strip minutes and seconds
results['time'] = results.Time.apply(lambda x: datetime.datetime.strptime(x,'%M:%S'))
# create time delta by subtracting a base time
results['timedelta'] = results.time - datetime.datetime.strptime('00:00,0','%M:%S,%f')
# create a seconds column
results['secs'] = results['timedelta'].apply(lambda x: x / np.timedelta64(1, 's'))

results.head()

,first,last,gender,age,clubname,open,master,agegrade,Name,Place,No.,Age,S,City,St,Time,Pace,time,timedelta,secs
0,Katherine,Ambrosio,F,65,NaN,yes,yes,yes,Katherine Ambrosio,230,201,65,F,Delmar,NY,38:10,9:33,1900-01-01 00:38:10,00:38:10,2290
1,Marie,Arrao,F,60,NaN,yes,yes,yes,Marie Arrao,235,202,60,F,Clifton Park,NY,38:19,9:35,1900-01-01 00:38:19,00:38:19,2299
2,Amy,Babeu,F,23,ARE Racing Team,yes,no,no,Amy Babeu,51,203,23,F,Guilderland,NY,26:02,6:31,1900-01-01 00:26:02,00:26:02,1562
3,Richard,Baker,M,41,NaN,yes,yes,no,Richard Baker,260,206,41,M,Rensselaer,NY,41:01,10:16,1900-01-01 00:41:01,00:41:01,2461
4,Richard,Baker,M,41,NaN,yes,yes,no,Richard Baker,260,206,41,M,Rensselaer,NY,41:01,10:16,1900-01-01 00:41:01,00:41:01,2461


In [255]:
# get unique club names
clubnames=clubs.clubname.unique()
clubnames = [x for x in clubnames if str(x) != 'nan']
print clubnames

['ARE Racing Team', 'Hudson Mohawk Road Runners Club', 'Kinetic Running Club', 'Willow Street AC', 'Team Utopia', 'Adirondack Athletic Club', 'Saratoga Stryders', 'Fleet Feet Sports Running/Walking Club', 'Kinderhook Runners Club', 'Castleton Cruisers']


In [256]:
def clubReport(clubname, gender, category):
    
    racers=results[results.clubname==clubname]
    racers=racers[racers.gender==gender]
    racers=racers[racers[category]=='yes']
    #print racers
    '''males=males.sort('age')
    
    females=racers[racers.gender=='F']
    females=females.sort('age')
    
    mhtml = males.to_html(index=False) if males.shape[0]>0 else None
    fhtml = females.to_html(index=False) if females.shape[0]>0 else None
    
    return mhtml, fhtml'''
    return racers

In [257]:
def scoreCategory(gender, category):
    names=[]
    times=[]
    for club in clubnames:
        scored=clubReport(club, gender, category)
        
        if scored.shape[0]>=4:
            names.append(club)
            times.append(scored.timedelta.order()[0:4].sum())
    
    return pd.DataFrame({'club name':names,'total_time':times}).sort('total_time')
        

In [258]:
results.Name.astype(str).split(' ',1)[0]test = scoreCategory('M', 'master')
test

,club name,total_time
0,Willow Street AC,01:31:38
2,Adirondack Athletic Club,01:36:52
3,Saratoga Stryders,01:44:57
1,Team Utopia,01:45:27
4,Fleet Feet Sports Running/Walking Club,02:00:42


In [259]:
test=clubReport('Willow Street AC', 'M', 'open')
print test.shape[0]>=4
print test.timedelta.order()[0:4].sum()

True
0 days 01:25:04


In [260]:
f = open('../output/usatfp.html','w')

message = """<html>
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="chrome=1">
    <title>Runnin&#39; of the Green USATF Grand Prix</title>

    <link rel="stylesheet" href="stylesheets/styles.css">
    <link rel="stylesheet" href="stylesheets/github-light.css">
    <meta name="viewport" content="width=device-width">
    <!--[if lt IE 9]>
    <script src="//html5shiv.googlecode.com/svn/trunk/html5.js"></script>
    <![endif]-->
  </head>
<body><h1>Runnin' of the Green USATF Grand Prix Club Preliminary Results</h1>"""

#clubname='ARE Racing Team'

for category in ['open','master','agegrade']:
    for gender in ['M','F']:

        table=scoreCategory(gender, category)
        html=table.to_html(index=False)

        message=message+"<h2>"+gender+" "+category+"</h2>"

        if (html!=None):
            message=message+html
            message=message+"<br>"

        message=message+"""</body>
        </html>"""

f.write(message)
f.close()

In [261]:
from weasyprint import HTML
HTML("../output/usatfp.html").write_pdf("../output/rotg2016_usatfteam.pdf")

Ignored `overflow-x: auto` at 133:3, unknown property.
Ignored `-webkit-font-smoothing: subpixel-antialiased` at 164:3, unknown property.
Ignored `-webkit-font-smoothing: subpixel-antialiased` at 248:3, unknown property.
Parse error at 1:1, unknown at-rule in stylesheet context: @font-face
Parse error at 15:1, unknown at-rule in stylesheet context: @font-face
Parse error at 29:1, unknown at-rule in stylesheet context: @font-face
Parse error at 43:1, unknown at-rule in stylesheet context: @font-face
Parse error at 251:8, expected a media type, got IDENT, IDENT, (
Parse error at 286:8, expected a media type, got IDENT, IDENT, (
Parse error at 304:8, expected a media type, got IDENT, IDENT, (
